# MiniWorld VectorHash + CNN Performance Tests

This notebook runs a grid of VectorHash localization tests on the
MiniWorld-Maze environment, comparing raw-flattened pixels against a
ResNet-18 pretrained encoder.

In [ ]:
# ── Imports, device, reproducibility ──
import os, itertools, pickle
import torch, numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

from miniworld.params import DEFAULT_PARAMS
from preprocessing_cnn import PreprocessingCNN
from vectorhash import build_vectorhash_architecture
from miniworld_agent import MiniworldVectorhashAgent
from agent import kidnapping_test
from smoothing import (
    IdentitySmoothing,
    PolynomialSmoothing,
    SoftmaxSmoothing,
    RatSLAMSmoothing,
)
from shifts import RatShift

# fix seeds
torch.manual_seed(0)
np.random.seed(0)

device = torch.device("cpu")

2025-06-20 14:27:46.190 Python[8639:508622] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/zz/_8nvyjvj4jd9v1rfts46s0ph0000gn/T/com.apple.python3.savedState


In [ ]:
# ── Env builder, shapes & limits ──

shapes = [(3, 3, 4), (4, 4, 5)]


def make_env():
    params = DEFAULT_PARAMS.copy().no_random()
    env = gym.make(
        "MiniWorld-Maze-v0", max_episode_steps=-1, params=params, domain_rand=False
    )
    # compute limits from the wrapper attrs
    min_x = env.get_wrapper_attr("min_x")
    max_x = env.get_wrapper_attr("max_x")
    min_z = env.get_wrapper_attr("min_z")
    max_z = env.get_wrapper_attr("max_z")
    limits = torch.tensor(
        [max_x - min_x, max_z - min_z, 2 * np.pi], device=device
    ).float()
    return env, limits

In [ ]:
# ── CNN Preprocessor & Hyperparams ──

# 1) CNN encoder (ResNet-18 adapter)
cnn_preproc = PreprocessingCNN(
    device=device,
    latent_dim=128,
    input_channels=3,
    target_size=(224, 224),
    model_path="resnet18_adapter.pth",
)

# 2) Hyperparameter grid
store_opts = [True, False]  # True="Always", False="When New"
shift_opts = ["additive", "multiplicative"]
hard_opts = [True, False]  # True="Hard", False="Soft"
smooth_opts = [
    IdentitySmoothing(),
    PolynomialSmoothing(k=1.0),
    PolynomialSmoothing(k=1.5),
    SoftmaxSmoothing(T=0.1),
    RatSLAMSmoothing(device=device),
]

# 3) Output folder
basedir = "miniworld_cnn_tests"
os.makedirs(basedir, exist_ok=True)

/Users/awalnoorsinghbajaj/Desktop/rl-research/vectorhash/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/awalnoorsinghbajaj/Desktop/rl-research/vectorhash/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# ─── 1) Motion primitives (90° turns) ───
# MiniWorld-Maze: 0=turn left 90°, 1=turn right 90°, 2=move forward
forward_10 = [2] * 10  # forward ×10
forward_3 = [2] * 3  # forward ×3
turn_r = [1]  # one 90° right turn
turn_l = [0]  # one 90° left turn

# ─── 2) Build path ───
# Sequence: F10 → R → F3 → R → F10 → L → F3 → L → F10
path = (
    forward_10
    + turn_r
    + forward_3
    + turn_r
    + forward_10
    + turn_l
    + forward_3
    + turn_l
    + forward_10
)

# Check length
print("Path length:", len(path))  # should be 10+1+3+1+10+1+3+1+10 = 40

# ─── 3) Visibility flags ───
# Make a visibility segment for each chunk
visible_10 = [True] * 10
visible_3 = [True] * 3
visible_1 = [True] * 1
not_visible_1 = [False] * 1

# Now concatenate in the same order as `path`
visibles = (
    visible_10
    + visible_1  # turn_r
    + visible_3
    + visible_1  # turn_r
    + visible_10
    + visible_1  # turn_l
    + visible_3
    + visible_1  # turn_l
    + visible_10
)

# Quick check
print("Visibles length:", len(visibles))  # → 40

# ─── 4) Noise list ───
# Here we leave each step noise-free
noise_list = [[] for _ in path]

# Final sanity check
assert len(path) == len(visibles) == len(noise_list), "Lengths must match!"

Path length: 40
Visibles length: 40


In [ ]:
# ── Run CNN‐only kidnapping tests ──

for store_new, shift_m, hard_store, smooth in itertools.product(
    store_opts, shift_opts, hard_opts, smooth_opts
):
    # descriptive run name
    sm_str = (
        "Identity"
        if isinstance(smooth, IdentitySmoothing)
        else (
            f"Poly(k={smooth.k})"
            if hasattr(smooth, "k")
            else f"Softmax(T={smooth.T})" if hasattr(smooth, "T") else "RatSLAM"
        )
    )
    run_name = (
        f"cnn__{'Always' if store_new else 'WhenNew'}"
        f"__{shift_m}__{'Hard' if hard_store else 'Soft'}__{sm_str}"
    )
    print("Running", run_name)

    # 1) make env & limits
    env, limits = make_env()

    # 2) build VectorHash + agent
    vh = build_vectorhash_architecture(
        shapes=shapes,
        N_h=600,
        input_size=128,  # CNN latent size
        initalization_method="by_sparsity",
        limits=limits,
        device=device,
        shift=RatShift(device=device),
        smoothing=smooth,
    )
    agent = MiniworldVectorhashAgent(
        vectorhash=vh,
        env=env,
        hard_store=hard_store,
        store_new=store_new,
        shift_method=shift_m,
        preprocessor=cnn_preproc,
    )

    # 3) kidnapping history
    # signature: kidnapping_test(agent, path, visibles, limits, noise_dist=None)
    hist = kidnapping_test(agent, path, visibles, limits)

    # 4a) save history
    with open(f"{basedir}/{run_name}.pkl", "wb") as f:
        pickle.dump(hist, f)

    # 4b) plot error curve
    plt.figure(figsize=(6, 3))
    plt.plot(hist.errors, label=run_name)
    plt.title(run_name)
    plt.xlabel("Timestep")
    plt.ylabel("Position error")
    plt.legend(fontsize="x-small")
    plt.tight_layout()
    plt.savefig(f"{basedir}/{run_name}_error.png", dpi=120)
    plt.close()

    env.close()

Running cnn__Always__additive__Hard__Identity
Falling back to num_samples=4
Falling back to non-multisampled frame buffer
Falling back to num_samples=4
Falling back to non-multisampled frame buffer
by_sparsity
module shapes:  [(3, 3, 4), (4, 4, 5)]
N_g     :  116
N_patts :  2880
N_h     :  600
info for each h directly after learning it
h max, min, mean tensor(7.5059) tensor(0.) tensor(1.8728)
h_from_s max, min, mean tensor(7.5040) tensor(0.) tensor(1.8723)
h_from_s_denoised max, min, mean tensor(1235.0868) tensor(0.) tensor(166.9291)
avg nonzero/greaterzero h from book: tensor(543) tensor(543)
avg nonzero/greaterzero h from s: tensor(543) tensor(543)
avg nonzero/greaterzero h from s denoised: tensor(323) tensor(323)
mse/cosinesimilarity h from book and h from s tensor(3.1806e-07) tensor([1.0000])
mse/cosinesimilarity h from book and h from s denoised tensor(80205.8750) tensor([0.5578])
mse/cosinesimilarity s and s from h from s tensor(3.0940e-10) tensor([1.])
mse/cosinesimilarity s and

/Users/awalnoorsinghbajaj/Desktop/rl-research/vectorhash/hippocampal_sensory_layers.py:110: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:4416.)
  1 + input.T @ self.inhibition_matrix_hs @ input
/Users/awalnoorsinghbajaj/Desktop/rl-research/vectorhash/vectorhash.py:102: UserWarning: Using a target size (torch.Size([1, 600])) that is different to the input size (torch.Size([600])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  torch.nn.functional.mse_loss(h, h_from_s),
/Users/awalnoorsinghbajaj/Desktop/rl-research/vectorhash/vectorhash.py:109: UserWarning: Using a target size (torch.Siz

IndexError: index 12 is out of bounds for dimension 0 with size 12